# How to use add breakpoints using interrupt?

!!! tip "Prerequisits"

    This guide assumes familiarity with the following concepts:

    * [Breakpoints](../../../concepts/breakpoints)
    * [LangGraph Glossary](../../../concepts/low_level)
    

An `interrupt` is a convenient way to support human-in-the-loop workflows.

To use an `interrupt`, you must enable a checkpointer, as the feature relies on persisting the graph state.

An `interrupt` can be used within a node to pause execution and wait for input, as shown in this example:

```python
async def some_node(state: State):
    ...
    # Surface any value as part of the interrupt
    value = {"question": "how old are you?"} 
    answer = interrupt(value)
    ...
```

Graph execution will pause when the interrupt function is called. To resume execution, pass a `Command` with the desired resume value:

```python
for chunk in graph.stream(Command(resume=some_value), config={"configurable": {"thread_id": ...}}):
    ...
```

Remember that graph execution always restarts at the beginning of the node. Be cautious of side effects, such as API calls that mutate data, as these may inadvertently be triggered multiple times.

When a node contains multiple interrupt calls, LangGraph maintains a list of resume values scoped to the specific task executing the node. When resuming, execution always starts at the beginning of the node, and for each interrupt encountered, LangGraph checks whether a corresponding value exists in the task's list. Matching is strictly index-based, making the order of interrupt calls within the node critical. Users should avoid logic that dynamically removes, adds, or reorders interrupt calls between executions, as this can lead to mismatched indices. Such patterns often involve unconventional state mutations, such as altering state via `Command(resume=..., update=SOME_STATE_MUTATION)` or relying on global variables to modify the node's structure.

## Setup

First we need to install the required packages:

In [14]:
%%capture --no-stderr
%pip install --quiet -U langgraph

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Basic usage of interrupt and Command

Here is an example that shows how to use `interrupt` to interrupt the execution of a graph, and then resume the execution using the `Command` primitive.

In [10]:
import uuid
import operator
from typing import TypedDict, Annotated, Optional

from langgraph.graph import StateGraph
from langgraph.constants import START, INTERRUPT
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    """The graph state."""

    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""


counter = 0


def node(state: State):
    global counter
    counter += 1
    print(f"> Entered the node: {counter} # of times")
    answer = interrupt(
        # This value will be sent to the client
        # as part of the interrupt information.
        "what is your age?"
    )
    print(f"> Received an input from the interrupt: {answer}")
    return {"human_value": answer}


builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

# A checkpointer must be enabled for interrupts to work!
checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)

> Entered the node: 1 # of times
{'__interrupt__': (Interrupt(value='what is your age?', resumable=True, ns=['node:62e598fa-8653-9d6d-2046-a70203020e37'], when='during'),)}


Let's resume graph execution from the given node:

In [11]:
command = Command(resume="some input from a human!!!")

for chunk in graph.stream(Command(resume="some input from a human!!!"), config):
    print(chunk)

> Entered the node: 2 # of times
> Received an input from the interrupt: some input from a human!!!
{'node': {'human_value': 'some input from a human!!!'}}


!!! important "Graph execution resumes at the start of a node"

    Graph execution resumes from the start of the **node** where the interrupt was raised rather than from the line where the `interrupt` was raised.

    As a result, you should see that the node was entered 2 times rather than once!

    Exercise care if your code has side-effects like making mutable API calls between consecutive interrupts!

## Using multiple interrupts calls within a single node

In some situations, you may need to use interrupt more than once within a single node. A common use case is performing runtime validation on the value supplied through `Command(resume=value)`.

In [12]:
import uuid
import operator
from typing import TypedDict, Annotated, Optional, Literal

from langgraph.graph import StateGraph
from langgraph.constants import START
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    """The graph state."""

    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""


counter = 0


def node(state: State):
    global counter
    counter += 1
    print(f"> Entered the node: {counter} # of times")

    answer = None
    question = "What is your age?"

    while answer is None:
        answer = interrupt(question)

        if not isinstance(answer, int) or answer < 0:
            question = f"'{answer} is not a valid age. What is your age?"
            answer = None
            continue
        else:
            break

    return {"human_value": f"The human is {answer} years old."}


builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

# A checkpointer must be enabled for interrupts to work!
checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)

> Entered the node: 1 # of times
{'__interrupt__': (Interrupt(value='What is your age?', resumable=True, ns=['node:ed4d470f-5753-d7f3-eec6-4435f5f93f72'], when='during'),)}


Let's resume with a bad input

In [13]:
bad_input = -20  # Negative number!
for chunk in graph.stream(Command(resume=bad_input), config):
    print(chunk)

> Entered the node: 2 # of times
{'__interrupt__': (Interrupt(value="'-20 is not a valid age. What is your age?", resumable=True, ns=['node:ed4d470f-5753-d7f3-eec6-4435f5f93f72'], when='during'),)}


In [14]:
bad_input = {"foo": "bar"}  # Not a number!
for chunk in graph.stream(Command(resume=bad_input), config):
    print(chunk)

> Entered the node: 3 # of times
{'__interrupt__': (Interrupt(value="'{'foo': 'bar'} is not a valid age. What is your age?", resumable=True, ns=['node:ed4d470f-5753-d7f3-eec6-4435f5f93f72'], when='during'),)}


In [15]:
ok_input = 25
for chunk in graph.stream(Command(resume=ok_input), config):
    print(chunk)

> Entered the node: 4 # of times
{'node': {'human_value': 'The human is 25 years old.'}}


## Usage with invoke / ainvoke

If you're using `invoke` and/or `ainvoke`, you will need to explicitly access the state of the graph using `graph.get_state(config)` to determine if there was an interrupt and if so what value it was associated with.

In [21]:
import uuid
import operator
from typing import TypedDict, Annotated, Optional

from langgraph.graph import StateGraph
from langgraph.constants import START, INTERRUPT
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    """The graph state."""

    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""


counter = 0


def node(state: State):
    global counter
    counter += 1
    print(f"> Entered the node: {counter} # of times")
    answer = interrupt(
        # This value will be sent to the client
        # as part of the interrupt information.
        "what is your age?"
    )
    print(f"> Received an input from the interrupt: {answer}")
    return {"human_value": answer}


builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

# A checkpointer must be enabled for interrupts to work!
checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

for event in graph.invoke({"foo": "abc"}, config):
    print

> Entered the node: 1 # of times


In [22]:
state = graph.get_state(config)

print(state.next)
print()
print(state.tasks[0])
print()
print(state.tasks[0].interrupts)

('node',)

PregelTask(id='57efb8b4-1170-b872-647e-60de07598d61', name='node', path=('__pregel_pull', 'node'), error=None, interrupts=(Interrupt(value='what is your age?', resumable=True, ns=['node:57efb8b4-1170-b872-647e-60de07598d61'], when='during'),), state=None, result=None)

(Interrupt(value='what is your age?', resumable=True, ns=['node:57efb8b4-1170-b872-647e-60de07598d61'], when='during'),)


Let's resume now:

In [23]:
ok_input = 25
graph.invoke(Command(resume=ok_input), config)

> Entered the node: 2 # of times
> Received an input from the interrupt: 25


{'foo': 'abc', 'human_value': 25}